In [33]:
import json
import foursquareWithHosts
with open('keys/appCredentials.json') as credentialsFile:    
    credentials = json.load(credentialsFile)
with open('keys/accessTokens.json') as tokensFile:    
    accessTokens = json.load(tokensFile)

In [52]:
clients = []
users = []
for accessToken in accessTokens[2:3]:
    client = foursquareWithHosts.Foursquare(client_id=credentials['client_id'], client_secret=credentials['client_secret'], redirect_uri=credentials['redirect_uri'])
    client.set_access_token(accessToken)
    user = client.users()
    clients.append(client)
    users.append(user)

Hi


In [ ]:
from datetime import datetime
from sys import stdout
def extractComponents(createdAt, tzOffset):
    date = datetime.utcfromtimestamp(createdAt+(tzOffset-60)*60)
    return date
import requests
requests.packages.urllib3.disable_warnings()

# Getting the data

In [53]:
import pandas as pd
usersDF = pd.DataFrame()
checkinsDF = pd.DataFrame()
checkpointsDF = pd.DataFrame()

allUsersIds = []
allCheckInsIds = []

currentVenueId = 0
currentBotId = 0

In [54]:
with open('keys/botIds.json') as botIdsFile:
    botIds = json.load(botIdsFile)
allUsersIds = botIds

In [55]:
#3.1 Savethe info to the DB
def parseHereNowDict(hereNowDict, venueId):
    usersInVenue = []
    currentCheckIns = []

    if hereNowDict['hereNow']:
        global checkpointsDF
        global usersDF
        global checkinsDF
        hereNowCount =  hereNowDict['hereNow']['count']
        checkpointsDF = checkpointsDF.append([{'createdAt':datetime.now(),
                                               'hereNow':hereNowCount, 
                                               'venueId':venueId}])

        print 'Here now:', hereNowCount
        print ''
        print 'real here now:', len(hereNowDict['hereNow']['items'])
        #print  hereNowDict['hereNow']

        #Get check-in info first
        for checkinAtVenue in hereNowDict['hereNow']['items']:
            if checkinAtVenue['type'] == 'checkin':
                if checkinAtVenue['id'] not in allCheckInsIds and checkinAtVenue['user']['id'] not in botIds:
                    currentCheckIns.append({'checkInId':checkinAtVenue['id'],
                                            'createdAt':checkinAtVenue['createdAt'],
                                            'createdAtDate':extractComponents(checkinAtVenue['createdAt'],checkinAtVenue['timeZoneOffset']),
                                            'venueId':venueId,
                                            'userId':checkinAtVenue['user']['id']
                                           })
                    allCheckInsIds.append(checkinAtVenue['id'])
                    #Now get the user
                    userInVenue =  checkinAtVenue['user']
                    #print 'User', userInVenue['id'],'Checked-in at:', extractComponents(checkinAtVenue['createdAt'],checkinAtVenue['timeZoneOffset'])
                    if userInVenue['id'] not in allUsersIds:
                        userInfo = {'userId':userInVenue['id']}
                        if 'gender' in userInVenue.keys() and userInVenue['gender']:
                            userInfo["gender"] = userInVenue['gender']
                        if 'firstName' in userInVenue.keys() and userInVenue['firstName']:
                            userInfo["firstName"] = userInVenue['firstName']
                        if 'lastName' in userInVenue.keys() and userInVenue['lastName']:
                            userInfo["lastName"] = userInVenue['lastName']
                        usersInVenue.append(userInfo)
                        allUsersIds.append(userInVenue['id'])
        if len(usersInVenue):
            usersDF = usersDF.append(usersInVenue)
        if len(currentCheckIns):
            checkinsDF = checkinsDF.append(currentCheckIns)

        usersDF.to_csv('data/25June/users.tsv', sep='\t',encoding='utf-8')
        checkinsDF.to_csv('data/25June/checkins.tsv', sep='\t',encoding='utf-8')
        checkpointsDF.to_csv('data/25June/checkPoints.tsv', sep='\t',encoding='utf-8')

In [56]:
def getNextCheckinInfo():
    global currentBotId
    global currentVenues
    global currentVenueId
    currentVenue = venuesForUsers[currentVenueId]
    try:
        checkIn = clients[currentBotId].checkins.add(currentVenue)
        hereNowDict = clients[currentBotId].venues.herenow(currentVenue['venueId'])
        parseHereNowDict(hereNowDict, currentVenue['venueId'])
        
        currentVenueId = (currentVenueId+1)%len(venuesForUsers)
        currentBotId = (currentBotId+1)%len(clients)
    except:
        print 'Failed'
    print currentBotId
    print currentVenueId

In [57]:
with open('data/top90healthVenuesSummer.json') as venuesFile:
    topHealtPlaces = json.load(venuesFile)
    
venuesForUsers = topHealtPlaces

# Run the loop

In [ ]:
import schedule
import time

schedule.clear()

timeInterval = 3*60/len(venuesForUsers)
schedule.every(1).minutes.do(getNextCheckinInfo)

while True:
    schedule.run_pending()
    time.sleep(1)